In [ ]:
# sample data is ready - Done 
# code to read the sample data and extracted chemicals -  done  
# code to convert raw md file to readble md file - done
# code to chunk the md file - chunk size 2000 tokens - done
# store the chemicals to flashtext - to be done 
# chemicals in the chunk - chunk wise chemicals for prompt buulding - done
# prompt input instruction 
# value_extraction_prompt - to be done 
# complete prompt building - to be done
# loop over chunks - other value extraction - to be done
    ## other value mapping to RR chemicals - to be done 
# evaluation scripts to evaluate columns wise - to be done
# test on multipule files - to be done

In [ ]:
# imports 
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re
from markdownify import markdownify as md
from bs4 import BeautifulSoup

In [ ]:
import logging
import traceback
import json
import pandas as pd

In [ ]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [ ]:

@staticmethod
def remove_html_comments(md_content):
    # This regex removes everything from <!-- to -->
    return re.sub(r'<!--.*?-->', '', md_content, flags=re.DOTALL)

@staticmethod
def convert_html_tables_to_markdowns(md_content):
    # Remove HTML comments
    md_content = remove_html_comments(md_content)

    new_md_content = md_content
    try:
        soup = BeautifulSoup(md_content, "html.parser")
        for table in soup.find_all("table"):
            html_str = str(table)
            markdown_table = md(html_str).strip()

            if html_str in new_md_content:
                new_md_content = new_md_content.replace(html_str, markdown_table)
            else:
                start_index = new_md_content.find("<table")
                end_index = new_md_content.find("</table>", start_index) + len("</table>")
                if start_index != -1 and end_index != -1:
                    new_md_content = (
                        new_md_content[:start_index]
                        + markdown_table
                        + new_md_content[end_index:]
                    )
    except Exception as e:
        logger.error(f"Error converting HTML tables to markdown: {str(e)}")
        logger.error(traceback.format_exc())

    return new_md_content.lstrip().rstrip()


In [ ]:
def creat_chunks(md_path):
    # load the file and chunking it 

    loader = TextLoader(md_path,encoding='utf-8')

    md_content = loader.load()
    
    md_content = md_content[0].page_content

    md_content = convert_html_tables_to_markdowns(md_content)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

    chunks = text_splitter.split_text(md_content)

    return chunks

In [ ]:
md_path = "data\\US_DEA_Source_file.md"
chunks = creat_chunks(md_path)

In [ ]:
# code to read extracted chemicals
import pandas as pd

df = pd.read_excel("data\\US_DEA_chemicals.xlsx", header=0)

df.columns = df.iloc[0]

In [ ]:
df.columns = df.iloc[0]

In [ ]:
extracted_chemicals = df['Chemical Name'].values.tolist()[1:]

In [ ]:
# flashtext chemical annotation 
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor(case_sensitive=False)

for chemical in extracted_chemicals:
    keyword_processor.add_keyword( chemical )




In [ ]:
# chunk wise available chemicals retrival
chemical_in_chunk = keyword_processor.extract_keywords(chunks[18])

In [ ]:
print(chunks[18])

In [ ]:
context_example = """
                            | --- | --- |
                            | (19) Pethidine (meperidine) | 9230 |
                            | (20) Pethidine-Intermediate-A, 4-cyano-1-methyl-4-phenylpiperidine | 9232 |
                            | (21) Pethidine-Intermediate-B, ethyl-4-phenylpiperidine-4-carboxylate | 9233 |
                            | (22) Pethidine-Intermediate-C, 1-methyl-4-phenylpiperidine-4-carboxylic acid | 9234 |
                            | (23) Phenazocine | 9715 |
                            | (24) Piminodine | 9730 |
                            | (25) Racemethorphan | 9732 |
                            | (26) Racemorphan | 9733 |
                            | (27) Remifentanil | 9739 |
                            | (28) Sufentanil | 9740 |
                            | (29) Tapentadol | 9780 |
                            | (30) Thiafentanil | 9729 |
                            
                """

value_format_prompt = """ 
                            [
                                {
                                    "Chemical Name": "Pethidine",
                                    "Type": "DEA Number",
                                    "Value": "9230",
                                    "remark" : "present"
                                },
                                {
                                    "Chemical Name": "Tapentadol",
                                    "Type": "DEA Number",
                                    "Value": "9780",
                                    "remark" : "Present"
                                },
                                {
                                    "Chemical Name": "Phenazocine",
                                    "Type": "DEA Number",
                                    "Value": "9715",
                                    "remark" : "Present"
                                },
                            ]
                            """

    # Example JSON format with corrected syntax
value_extract_example = f"""Output format (in JSON): Please format the output strictly as valid JSON, using double quotes (") around all keys and string values.
                            {value_format_prompt}
    
                        """

# Combined value extraction section
value_extraction = f"""Context: {context_example}
Values to extract: {value_extract_example}
"""

# Final prompt template
prompt_template = f"""You are a Chemical relational values extraction expert.

    Instructions for Extracting Other Values:

    You will be provided with list of chemicals and a context, from the given context, extract other values for every chemical given in the list. 
    
    Take the example given below as a refference. If value not exit then return None for that variable

    {value_extraction}

    """ 

In [ ]:
print(prompt_template)

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

In [ ]:
model_name = "gpt-4o-mini"
llm_deployment = "gpt-4o-mini"
api_version = "2024-12-01-preview"
embed_deployment = "text-embedding-3-small"
load_dotenv()

llm_client = AzureOpenAI(
                        api_version=api_version,
                        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                        api_key=os.getenv("OPENAI_API_KEY"),
                        azure_deployment = llm_deployment,
                        )


In [ ]:
chunk_text = chunks[18]
chunk_chemical = chemical_in_chunk
input_text = f""" 
list of chemical for which values to be extracted:
{chunk_chemical}

chunk text from which values to be extracted
{chunk_text}

"""

In [ ]:
print(input_text) 

In [ ]:
response = llm_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": prompt_template,
        },
        {
            "role": "user",
            "content": input_text,
        }
    ],
    max_tokens=4096,
    temperature=0.1,
    top_p=1.0,
    model=llm_deployment
)

In [ ]:
response_data = response.choices[0].message.content

In [ ]:
print(response_data)

In [ ]:
def other_value_extractions_from_llm(prompt_template, input_text):

        response = llm_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": prompt_template,
            },
            {
                "role": "user",
                "content": input_text,
            }
        ],
        max_tokens=4096,
        temperature=0.1,
        top_p=1.0,
        model=llm_deployment
                            )
        
        response_data = response.choices[0].message.content

        cleaned_response = response_data.replace("```json", "").replace("```", "").lstrip().rstrip()
        # Extra safety: Replace single quotes with double quotes (if clearly JSON-like)

        if cleaned_response.startswith("[{") and "'" in cleaned_response and '"' not in cleaned_response:
            cleaned_response = cleaned_response.replace("'", '"')
            
        chemicals_extracted = json.loads(cleaned_response)

        return chemicals_extracted

In [ ]:
def build_input_text(chunk_chemical,chunk_text):

    input_text = f""" 
    list of chemical for which values to be extracted:
    {chunk_chemical}

    chunk text from which values to be extracted
    {chunk_text}
    """
    return input_text

In [ ]:
chemical_other_values = []

for chunk in chunks:
    
    chemicals_in_chunk = keyword_processor.extract_keywords(chunk)

    if chemicals_in_chunk:

        input_text =  build_input_text(chemicals_in_chunk, chunk)
    
        other_values = other_value_extractions_from_llm(prompt_template, input_text)

        chemical_other_values.extend(other_values)
    
df = pd.DataFrame(chemical_other_values)



In [ ]:
df1 = pd.DataFrame(chemical_other_values)

# clean code

In [ ]:
import pandas as pd
from pre_processing import Pre_processing
from prompt_building import prompt_building
import re
from other_value_evaluation import ValuesComparision

md_path = "data\\CND_Ontaria\\cnd_ontario_dw.md"

extracted_chemical_path = "data\\CND_Ontaria\\list_6943_extracted_chemicals.xlsx"

ground_truth_path = "data\\CND_Ontaria\\list_6943_gt.xlsx"

compare_columns = ["value", "unit", "type", "remark", "item number", "listedunder"]

In [ ]:
# Inputs from user/UI
context_example = ""  # Example context for value format
value_format_prompt = [{}]  # List of JSON examples for value format

In [ ]:
chunks = Pre_processing.creat_chunks(md_path, chunk_size=2000)
print(f"chunks created total chunks {len(chunks)}")

In [ ]:
print(chunks[1])

In [ ]:
print(chunks[2])

In [ ]:
df = pd.read_excel(extracted_chemical_path, header=0) 

In [ ]:
df.shape[0]

In [ ]:
keyword_processor = Pre_processing.add_extracted_chemicas_to_flashtext(extracted_chemical_path)
print(f"added extrcated chemicals to keyword processor")

In [ ]:
prompt_template = prompt_building.build_prompt(context_example, value_format_prompt)

In [ ]:
print(prompt_template)

In [ ]:
chemicals_in_chunk = keyword_processor.extract_keywords(chunks[0])

In [ ]:
chemical_other_values = []

for i, chunk in enumerate(chunks):
    
    chemicals_in_chunk = keyword_processor.extract_keywords(chunk)

    chemicals_in_chunk = list(set(chemicals_in_chunk))

    if not chemicals_in_chunk:
        print(f"#### NO values extracted for the chunk number {i}, number of chemicals in the chunk 0")
        continue

    input_text = prompt_building.build_input_text(chemicals_in_chunk, chunk)

    extracted_values = prompt_building.other_value_extractions_from_llm(prompt_template, input_text)
    
    print(f">>>> chunk number {i} >>>>extracted chemicals {len(extracted_values)}>>>>actual chemicals {len(chemicals_in_chunk)} ")
    print(f"################ >>>>>>> missed chemicals {len(chemicals_in_chunk)-len(extracted_values)}>>>>>>>>################ ")

    chemical_other_values.extend(extracted_values)


In [ ]:
chemical_other_values

In [ ]:
df = pd.DataFrame(chemical_other_values)
# df.to_csv(extracted_csv_path, index=False)

In [ ]:
df

In [ ]:
df.to_excel("data\\A_SUSMP\\predicted_othervalues.xlsx",index=False)

In [ ]:
# merge the other values extracted with extracted chemicals
df1 = pd.read_excel(extracted_chemical_path,header=0)

# df1.columns = df1.iloc[0]  # First row becomes header
# df1 = df1[1:]              # Drop the first row from data
# df1 = df1.reset_index(drop=True)

In [ ]:
df1

In [ ]:
# Standardize both to lowercase
df1['Chemical Name'] = df1['Chemical Name'].str.lower()
df['Chemical Name'] = df['Chemical Name'].str.lower()

# Then merge
merged_df = pd.merge(df1, df, on='Chemical Name', how='left')


In [ ]:
# merged_df = pd.merge(df1, df, on='Chemical Name', how='left')

In [ ]:
merged_df

In [ ]:
merged_df.to_excel("data\\A_SUSMP\\merged_base_othervalues.xlsx",index=False)

In [ ]:
def map_othervalues_to_RR_chemicals(df):
    # Load input
    # df = pd.read_csv("inputrr1.csv")

    # Ensure consistent column order
    columns_to_copy = [col for col in df.columns if col not in ["CAS", "Chemical Name"]]

    # Split base and RR entries
    base_df = df[~df["CAS"].str.startswith("RR-")].copy()
    rr_df = df[df["CAS"].str.startswith("RR-")].copy()

    # Normalize names
    def normalize(name):
        return str(name).lower().strip()

    # Fill RR rows with corresponding base info using strict word-boundary matching
    for rr_idx, rr_row in rr_df.iterrows():
        rr_name = normalize(rr_row["Chemical Name"])
        
        for _, base_row in base_df.iterrows():
            base_name = normalize(base_row["Chemical Name"])
            
            # Use \b to ensure it matches whole words or with space boundaries
            if re.search(rf"\b{re.escape(base_name)}\b", rr_name):
                for col in columns_to_copy:
                    rr_df.at[rr_idx, col] = base_row.get(col, "")
                break  # Use first matching base chemical

    # Combine base + updated RR rows back
    final_df = pd.concat([base_df, rr_df], ignore_index=True)

    # # Save output
    # final_df.to_csv("output_with_base_data1.csv", index=False)
    # print(final_df)

    return final_df


In [ ]:
mapped_df = map_othervalues_to_RR_chemicals(merged_df)

In [ ]:
import numpy as np

# Replace only the string "None" with np.nan in the entire DataFrame
mapped_df.replace("None", np.nan, inplace=True)

In [ ]:
mapped_df

In [ ]:
mapped_df.to_excel("data\\A_SUSMP\\rr_mapped_other_values.xlsx", index=False)

In [ ]:
# pd.read_excel("data\\rr_mapped_other_values.xlsx")

In [ ]:
ground_truth_df = pd.read_excel(ground_truth_path)
ground_truth_df 

In [ ]:
ground_truth_df['Chemical Name'] = ground_truth_df['Chemical Name'].str.lower()

In [ ]:

evaluation_results = ValuesComparision.evaluate_extraction_accuracy(ground_truth_df, mapped_df, compare_columns)


# Compliled code

In [1]:
from pymongo import MongoClient

# # Example connection string (replace with yours)
# mongo_uri = "mongodb://172.203.242.247:27017"  # or from MongoDB Atlas
# client = MongoClient(mongo_uri)

# # List available databases
# print(client.list_database_names())

In [ ]:
# Select your database and collection
# db = client["chemadvisor-qa"]
# collections = db.list_collection_names()
# collections 

In [ ]:
from pymongo import MongoClient
from collections import defaultdict
import pprint
import pandas as pd

def infer_schema_from_collection(collection, sample_size=10):
    """Infer schema by inspecting sample documents from a MongoDB collection."""
    schema = defaultdict(set)

    sample_docs = collection.find().limit(sample_size)

    for doc in sample_docs:
        for key, value in doc.items():
            schema[key].add(type(value).__name__)  # record type name

    return dict(schema)

# Connect to MongoDB
mongo_uri = "mongodb://172.203.242.247:27017"  # or from MongoDB Atlas
client = MongoClient(mongo_uri) # Update with your URI

db = client["chemadvisor-qa"]
collection = db["section_data"]

# Get schema
schema = infer_schema_from_collection(collection)

# Print schema
print("📘 Inferred Schema:")
pprint.pprint(schema)


In [ ]:
# Fetch a sample document that contains 'sections_metadata'
sample_doc = collection.find_one({"sections_metadata": {"$exists": True, "$ne": []}})

if sample_doc:
    sections_metadata = sample_doc.get("sections_metadata", [])


In [ ]:
documents = list(collection.find())

In [2]:
def get_files_of_documents_avaiable():
    mongo_uri = "mongodb://172.203.242.247:27017"  # or from MongoDB Atlas
    client = MongoClient(mongo_uri) # Update with your URI

    db = client["chemadvisor-qa"]
    collection = db["section_data"]
    documents = list(collection.find())

    files_names = []
    for i in range(0,7):
        files_names.append(documents[i]['file_name'])

    del(documents)
    
    return files_names

In [3]:
files_names = get_files_of_documents_avaiable()
files_names

['U.S. - DEA (Drug Enforcement Administration) - Controlled Substances - Source.pdf',
 'EU - Military Common List - Source.pdf',
 'CELEX_02012R0649-20250301_EN_TXT.pdf',
 'Canada - CEPA - Source.pdf',
 'Canada - CEPA - ODS and Halocarbon Alternatives.pdf',
 'Odor Thresholds 3rd edition.pdf',
 'Australia - Standard for the Uniform Scheduling of Medicines and Poisons (SUSMP) - Schedule 5 - Source.pdf']

In [35]:

def get_list_ids_by_filename(file_name):
    # Find the document by file_name
    # file_name = "EU - Military Common List - Source.pdf"
    
    mongo_uri = "mongodb://172.203.242.247:27017"  # or from MongoDB Atlas
    client = MongoClient(mongo_uri) # Update with your URI

    db = client["chemadvisor-qa"]
    collection = db["section_data"]

    docs = list(collection.find({"file_name": file_name}))

    # Collect all ListIDs from sections_metadata
    list_ids = []

    for doc in docs:
        sections = doc.get('sections_metadata', [])
        for section in sections:
            list_id = section.get('ListID')
            if list_id is not None:
                list_ids.append(list_id)

    return list_ids


In [36]:
file_name = 'CELEX_02012R0649-20250301_EN_TXT.pdf'
get_list_ids_by_filename(file_name)

[1754]

In [6]:
def get_section_md_data_for_list_id(lisd_id):
    
    mongo_uri = "mongodb://172.203.242.247:27017"  # or from MongoDB Atlas
    client = MongoClient(mongo_uri) # Update with your URI

    db = client["chemadvisor-qa"]
    collection = db["section_data"]

    results = list(collection.find({
        "sections_metadata.ListID": lisd_id}))

    for doc in results:
        print(f"\n📄 File: {doc.get('file_name')}")
        matched_sections = [ sec for sec in doc.get("sections_metadata", []) if sec.get("ListID") == lisd_id ]

    md_data = matched_sections[0]['matched_section']
    
    return md_data


In [37]:
data = get_section_md_data_for_list_id(6611)


📄 File: EU - Military Common List - Source.pdf


In [39]:
print(data)



<!-- PageNumber="C 72/11" -->
<!-- PageHeader="Official Journal of the European Union" -->
<!-- PageHeader="EN" -->
<!-- PageHeader="28.2.2023" -->

2\. Any substance listed in the ML8 sub-items is subject to this list, even when utilised in an application other than that
indicated. (e.g. TAGN is predominantly used as an explosive but can also be used either as a fuel or an oxidizer.)

3\. For the purposes of ML8, particle size is the mean particle diameter on a weight or volume basis. International or equivalent
national standards will be used in sampling and determining particle size.

a. 'Explosives' as follows, and 'mixtures' thereof:
☒

1\. ADNBF (aminodinitrobenzofuroxan or 7-amino-4,6-dinitrobenzofurazane-1-oxide) (CAS 97096-78-1);

2\. BNCP (cis-bis (5-nitrotetrazolato) tetra amine-cobalt (III) perchlorate)
(CAS 117412-28-9);

3\. CL-14 (diamino dinitrobenzofuroxan or 5,7-diamino-4,6-dinitrobenzofurazane-1-oxide ) (CAS
117907-74-1);

4\. CL-20 (HNIW or Hexanitrohexaazaisowurt

In [9]:
import pandas as pd

In [21]:
def get_ground_truth_data(list_id):
    
    mongo_uri = "mongodb://172.203.242.247:27017"  # or from MongoDB Atlas
    client = MongoClient(mongo_uri) # Update with your URI

    db = client["chemadvisor-qa"]
    collection_ld = db["list_data"]

    results = list(collection_ld.find({"ListID": list_id}))

    df = pd.DataFrame(results[0]['list_chemicals'])

    base_df = df[~df["CAS"].str.startswith("RR-")].copy()
    extracted_chemicals = base_df[['CAS','Chemical Name']]

    return base_df, extracted_chemicals

In [12]:
from pre_processing import Pre_processing

In [69]:
list_id = 1754
chunks = Pre_processing.creat_chunks(list_id, chunk_size=2000)


📄 File: CELEX_02012R0649-20250301_EN_TXT.pdf


In [70]:
len(chunks)

33

In [73]:
print(chunks[5])

| Chemical | CAS No | EC No | CN code ( \*\*\* ) | Subcategory (\*) | Use limitation ( \*\* ) | Countries for which no notification is required |
| --- | --- | --- | --- | --- | --- | --- |
| Azinphos-ethyl (+) | 2642-71-9 | 220-147-6 | ex 2933 99 80 | p(1)-p(2) | b-b |  |
| Azinphos-methyl (\*) | 86-50-0 | 201-676-1 | ex 2933 92 00 | p(1) | b |  |
| Azocyclotin (+) | 41083-11-8 | 255-209-1 | ex 2933 99 80 | p(1) | b |  |
| Benalaxyl (+) | 71626-11-4 | 275-728-7 | ex 2924 29 70 | p(1) | b |  |
| M9 Bendiocarb (+) | 22781-23-3 | 245-216-8 | ex 2932 99 00 | p(2) | b |  |
| Benfluralin (+) | 1861-40-1 | 217-465-2 | ex 2921 43 00 | p(1) | b |  |
| VM7 Benfuracarb (+) | 82560-54-1 |  | ex 2932 99 00 | p(1) | b |  |
| Bensultap | 17606-31-4 |  | ex 2930 90 98 | p(1)-p(2) | b-b |  |
| M9 Benthiavalicarb (+) | 413615-35-7 177406-68-7 |  | ex 2934 20 80 | p(1) | b |  |
| M7 Benzene (2) | 71-43-2 | 200-753-7 | ex 2902 20 00 ex 2707 10 00 | i(2) | sr |  |
| Benzene as a constituent of other subst

In [50]:
from database_client import db_data_extraction
import pandas as pd

In [77]:
list_id = 470

ground_truth_df, extracted_chemicals_df = db_data_extraction.get_ground_truth_data(list_id)

chunks = Pre_processing.creat_chunks(list_id, chunk_size=2000)
print(f"chunks created total chunks {len(chunks)}")

# df = pd.read_excel(extracted_chemical_path, header=0)



📄 File: Odor Thresholds 3rd edition.pdf
chunks created total chunks 50


In [89]:
print(chunks[29])

Table 6.1- Odor Threshold Values, cont.

| Copyright AIHA® For personal use only. Do not distribute. | # | Compound Name CAS Number Formula Molecular Weight | Range of Odor Values (ppm) | Odor Character | OSHA PEL (ppm) | ACGIH TLV® (ppm) | NIOSH REL (ppm) | DFG MAK (ppm) | OARS WEEL® (ppm) | EU OEL (ppm) |
| 69 | Cyanogen 460-19-5 NCCN 52.04 | >500 | almonds, pungent | — | C=5 | TWA = 10 | TWA = 5 STEL = 10 H | — | — |
| 70 | Cyanogen Chloride 506-77-4 ClCN 61.47 | 0.994 | acid, irritating | — | C=0.3 | C=0.3 | — | — | — |
| 71 | Cyclohexane 110-82-7 C6H12 84.16 | 0.52 - 784 | aromatic, chloroform, pungent, sweet | TWA = 300 | TWA = 100 | TWA = 300 | TWA = 200 STEL =800 | — | TWA =200 |
| 72 | Cyclohexanol 108-93-0 C6H11OH 100.16 | 0.058 - 0.491 | camphorous, menthol, phenol | TWA = 50 | TWA = 50 Skin | TWA =50 Skin | H | — | — |
| 73 | Cyclohexanone 108-94-1 6H100 98.14 | 0.052 - 219 | acetone, minty, sharp, sweet | TWA = 50 | TWA = 20 Skin | TWA = 25 Skin | H | — | TWA = 10 STEL = 2

In [ ]:
#470

In [94]:
list_id = 470
base_df, extracted_chemicals = get_ground_truth_data(list_id)

In [ ]:
base_df.head(10) 

,CAS,Chemical Name,value,unit,type,remark,listedunder,Formula
0,50-00-0,Formaldehyde,0.033,mg/m3,odor threshold value,recognizable,NaN,CH2O
1,50-00-0,Formaldehyde,0.027,ppm,odor threshold value,recognizable,NaN,CH2O
2,54-11-5,Nicotine,0.066,mg/m3,odor threshold value,NaN,NaN,C10H14N2
3,54-11-5,Nicotine,0.0099,ppm,odor threshold value,NaN,NaN,C10H14N2
4,56-23-5,Carbon tetrachloride,10.58,mg/m3,odor threshold value,NaN,NaN,CCl4
5,56-23-5,Carbon tetrachloride,1.68,ppm,odor threshold value,NaN,NaN,CCl4
6,57-06-7,Allyl isothiocyanate,0.037,mg/m3,odor threshold value,detectable,NaN,C4H5NS
7,57-06-7,Allyl isothiocyanate,0.0091,ppm,odor threshold value,detectable,NaN,C4H5NS
8,57-14-7,"1,1-Dimethylhydrazine",<0.75,mg/m3,odor threshold value,NaN,NaN,C2H8N2
9,57-14-7,"1,1-Dimethylhydrazine",<0.31,ppm,odor threshold value,NaN,NaN,C2H8N2


In [97]:
base_df.columns[1]

'Chemical Name'

In [26]:
from database_client import db_data_extraction

ground_truth_df, extracted_chemicals_df = db_data_extraction.get_ground_truth_data(list_id)

In [33]:

columns_compare = list(ground_truth_df.columns[2:])
if 'Formula' in columns_compare:
    columns_compare.remove('Formula')
columns_compare

['type', 'value', 'remark', 'listedunder']

In [ ]:
context = """ 
Table 6.1- Odor Threshold Values, cont.

| Copyright AIHA® For personal use only. Do not distribute. | # | Compound Name CAS Number Formula Molecular Weight | Range of Odor Values (ppm) | Odor Character | OSHA PEL (ppm) | ACGIH TLV® (ppm) | NIOSH REL (ppm) | DFG MAK (ppm) | OARS WEEL® (ppm) | EU OEL (ppm) |
| 69 | Cyanogen 460-19-5 NCCN 52.04 | >500 | almonds, pungent | — | C=5 | TWA = 10 | TWA = 5 STEL = 10 H | — | — |
| 70 | Cyanogen Chloride 506-77-4 ClCN 61.47 | 0.994 | acid, irritating | — | C=0.3 | C=0.3 | — | — | — |
| 71 | Cyclohexane 110-82-7 C6H12 84.16 | 0.52 - 784 | aromatic, chloroform, pungent, sweet | TWA = 300 | TWA = 100 | TWA = 300 | TWA = 200 STEL =800 | — | TWA =200 |
| 72 | Cyclohexanol 108-93-0 C6H11OH 100.16 | 0.058 - 0.491 | camphorous, menthol, phenol | TWA = 50 | TWA = 50 Skin | TWA =50 Skin | H | — | — |
| 73 | Cyclohexanone 108-94-1 6H100 98.14 | 0.052 - 219 | acetone, minty, sharp, sweet | TWA = 50 | TWA = 20 Skin | TWA = 25 Skin | H | — | TWA = 10 STEL = 20 |
| 74 | Cyclohexene 110-83-8 CH10 82.14 | 0.18 | sweet | TWA = 300 | TWA = 300 | TWA = 300 | — | — | — |
| 75 | Cyclohexylamine 108-91-8 C6HTINH2 99.17 | 2.42 | amine, ammonia, fishy | — | TWA = 10 | TWA = 10 | TWA = 2 STEL = 4 C=5 | — | — |
"""

In [ ]:
[
{
  'Chemical Name': 'Cyclohexylamine',
  'value': '9.83',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': 'detectable',
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyclohexylamine',
  'value': '2.42',
  'unit': 'ppm',
  'type': 'odor threshold value',
  'remark': 'detectable',
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyclohexanol',
  'value': '0.24',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyclohexanol',
  'value': '0.058',
  'unit': 'ppm',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyclohexanone',
  'value': '0.21',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyclohexanone',
  'value': '0.052',
  'unit': 'ppm',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyclohexane',
  'value': '1.8',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyclohexane',
  'value': '0.52',
  'unit': 'ppm',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyanogen',
  'value': '>533',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
  },
 {
  'Chemical Name': 'Cyanogen',
  'value': '>500',
  'unit': 'ppm',
  'type': 'odor threshold value',
  'remark': "None",
  'listedunder': "None",
 }
 ]


In [92]:
lst = ["Cyanogen", "Cyanogen Chloride", "Cyclohexane", "Cyclohexanol", "Cyclohexanone", "Cyclohexylamine"]

In [93]:
base_df[base_df['Chemical Name'].isin(lst)].to_dict(orient='records')

[{'CAS': '108-91-8',
  'Chemical Name': 'Cyclohexylamine',
  'value': '9.83',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': 'detectable',
  'listedunder': nan,
  'Formula': 'C6H13N'},
 {'CAS': '108-91-8',
  'Chemical Name': 'Cyclohexylamine',
  'value': '2.42',
  'unit': 'ppm',
  'type': 'odor threshold value',
  'remark': 'detectable',
  'listedunder': nan,
  'Formula': 'C6H13N'},
 {'CAS': '108-93-0',
  'Chemical Name': 'Cyclohexanol',
  'value': '0.24',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': nan,
  'listedunder': nan,
  'Formula': 'C6H12O'},
 {'CAS': '108-93-0',
  'Chemical Name': 'Cyclohexanol',
  'value': '0.058',
  'unit': 'ppm',
  'type': 'odor threshold value',
  'remark': nan,
  'listedunder': nan,
  'Formula': 'C6H12O'},
 {'CAS': '108-94-1',
  'Chemical Name': 'Cyclohexanone',
  'value': '0.21',
  'unit': 'mg/m3',
  'type': 'odor threshold value',
  'remark': nan,
  'listedunder': nan,
  'Formula': 'C6H10O'},
 {'CAS': '108-94-1',
  